<a href="https://colab.research.google.com/github/ruchitha2912/Doc2Graph/blob/main/AIRIW_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF

In [ ]:
import fitz  # PyMuPDF
import re

def extract_text_from_pdf(pdf_file_path):
    text = ""
    with fitz.open(pdf_file_path) as pdf_document:
        for page_number in range(pdf_document.page_count):
            page = pdf_document.load_page(page_number)
            page_text = page.get_text()
            text += page_text
    return text

# Function to replace "theta" with "ti" and "sigma" with "tt" using regular expressions
def replace_misinterpretations(text):
    text = re.sub(r'\b(Ɵ)\b', 'ti', text)
    text = re.sub(r'(\b\w*)Ɵ(\w*\b)', r'\1ti\2', text)
    text = re.sub(r'\b(Ʃ)\b', 'tt', text)
    text = re.sub(r'(\b\w*)Ʃ(\w*\b)', r'\1tt\2', text)
    return text

# Example usage:
pdf_file_path = "Natural language processing.pdf"  # Replace with the path to your PDF file
extracted_text = extract_text_from_pdf(pdf_file_path)
processed_text = replace_misinterpretations(extracted_text)
print(processed_text)


Natural language processing (NLP) is an interdisciplinary subfield of computer 
science and linguistics. It is primarily concerned with giving computers the ability to support and 
manipulate human language. It involves processing natural language datasets, such as text 
corpora or speech corpora, using either rule-based or probabilistic (i.e. statistical and, most 
recently, neural network-based) machine learning approaches. The goal is a computer capable 
of "understanding" the contents of documents, including the contextual nuances of the language 
within them. The technology can then accurately extract information and insights contained in the 
documents as well as categorize and organize the documents themselves. 
Challenges in natural language processing frequently involve speech recognition, natural-
language understanding, and natural-language generation. 
 
History 
Natural language processing has its roots in the 1940s. Already in 1940, Alan Turing published 
an article title

In [ ]:
!pip install transformers wikipedia newspaper3k GoogleNews pyvis

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import wikipedia
from newspaper import Article, ArticleException
from GoogleNews import GoogleNews
import IPython
from pyvis.network import Network

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

In [ ]:
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [ ]:
def from_small_text_to_kb(text, verbose=False):
    kb = KB()

    # Tokenizer text
    model_inputs = tokenizer(text, max_length=512, padding=True, truncation=True,
                            return_tensors='pt')
    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate
    gen_kwargs = {
        "max_length": 216,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3
    }
    generated_tokens = model.generate(
        **model_inputs,
        **gen_kwargs,
    )
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb
    for sentence_pred in decoded_preds:
        relations = extract_relations_from_model_output(sentence_pred)
        for r in relations:
            kb.add_relation(r)

    return kb

In [ ]:
kb = from_small_text_to_kb(processed_text, verbose=True)
kb.print()

Num tokens: 512
Relations:
  {'head': 'Computing Machinery and Intelligence', 'type': 'author', 'tail': 'Alan Turing'}
  {'head': 'natural language', 'type': 'studied by', 'tail': 'linguistics'}
  {'head': 'linguistics', 'type': 'studies', 'tail': 'natural language'}
  {'head': 'Turing test', 'type': 'discoverer or inventor', 'tail': 'Alan Turing'}


In [ ]:
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [ ]:
def from_text_to_kb(text, span_length=128, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [ ]:
kb = from_text_to_kb(processed_text, verbose=True)
kb.print()

Token indices sequence length is longer than the specified maximum sequence length for this model (2770 > 1024). Running this sequence through the model will result in indexing errors


Input has 2770 tokens
Input has 22 spans
Span boundaries are [[0, 128], [125, 253], [250, 378], [375, 503], [500, 628], [625, 753], [750, 878], [875, 1003], [1000, 1128], [1125, 1253], [1250, 1378], [1375, 1503], [1500, 1628], [1625, 1753], [1750, 1878], [1875, 2003], [2000, 2128], [2125, 2253], [2250, 2378], [2375, 2503], [2500, 2628], [2625, 2753]]
Relations:
  {'head': 'natural language', 'type': 'studied by', 'tail': 'linguistics', 'meta': {'spans': [[0, 128]]}}
  {'head': 'speech corpora', 'type': 'subclass of', 'tail': 'natural language', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Natural language processing', 'type': 'subclass of', 'tail': 'computer', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Natural language processing', 'type': 'subclass of', 'tail': 'linguistics', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Computing Machinery and Intelligence', 'type': 'author', 'tail': 'Alan Turing', 'meta': {'spans': [[125, 253]]}}
  {'head': 'Turing test', 'type': 'named after', 'tail': '

In [ ]:
!pip install py2neo

In [ ]:
def convert_to_rdf(kb):
    rdf_triples = []
    for relation in kb.relations:
        subject = relation['head']
        subject = subject.replace("'","")
        predicate = relation['type']
        predicate = predicate.replace(" ", "_")
        object_ = relation['tail']
        object_ = object_.replace("'","")
        rdf_triples.append((subject, predicate, object_))
    return rdf_triples

# Convert KB relations to RDF triples
rdf_triples = convert_to_rdf(kb)

In [ ]:
rdf_triples

[('natural language', 'studied_by', 'linguistics'),
 ('speech corpora', 'subclass_of', 'natural language'),
 ('Natural language processing', 'subclass_of', 'computer'),
 ('Natural language processing', 'subclass_of', 'linguistics'),
 ('Computing Machinery and Intelligence', 'author', 'Alan Turing'),
 ('Turing test', 'named_after', 'Alan Turing'),
 ('Georgetown experiment', 'point_in_time', '1954'),
 ('Chinese room experiment', 'named_after', 'John Searle'),
 ('Chinese room experiment', 'discoverer_or_inventor', 'John Searle'),
 ('ALPAC report', 'publication_date', '1966'),
 ('ALPAC', 'publication_date', '1966'),
 ('ALPAC report in 1966', 'publication_date', '1966'),
 ('SHRDLU', 'developer', 'Joseph Weizenbaum'),
 ('ELIZA', 'developer', 'Joseph Weizenbaum'),
 ('MARGIE', 'publication_date', '1975'),
 ('Ross Quillian', 'field_of_work', 'natural language'),
 ('MARGIE', 'developer', 'Schank'),
 ('PARRY', 'instance_of', 'chatterbot'),
 ('PARRY', 'instance_of', 'chatterbots'),
 ('PAM', 'publi

In [ ]:
!pip install neo4j

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt

# def construct_graph_from_rdf(rdf_triples):
#     # Create a directed graph
#     G = nx.DiGraph()

#     # Iterate through RDF triples and add edges to the graph
#     for triple in rdf_triples:
#         subject, predicate, object_ = triple
#         # Add edges to the graph
#         G.add_edge(subject, object_, label=predicate)

#     return G

# # Construct the graph from RDF triples
# graph = construct_graph_from_rdf(rdf_triples)

# # Draw the graph
# plt.figure(figsize=(16, 16))
# pos = nx.spring_layout(graph, seed=42)  # Positions for all nodes
# nx.draw(graph, pos, with_labels=True, node_size=2000, node_color="skyblue", font_size=12, font_weight="bold", arrowsize=20)
# edge_labels = nx.get_edge_attributes(graph, "label")
# nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_color="red")

# # Show the graph
# plt.title("Knowledge Graph")
# plt.show()


In [ ]:
from py2neo import Graph

In [ ]:
!pip3 install neo4j-driver

In [ ]:
# python3 example.py

from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://100.27.33.53:7687",
  auth=basic_auth("neo4j", "basements-injector-welder"))

# cypher_query = '''
# MATCH (n)
# RETURN COUNT(n) AS count
# LIMIT $limit
# '''

# with driver.session(database="neo4j") as session:
#   results = session.read_transaction(
#     lambda tx: tx.run(cypher_query, limit=10).data())
#   for record in results:
#     print(record['count'])

def execute_query(tx, query):
    tx.run(query)

# Add RDF triples to Neo4j
with driver.session(database="neo4j") as session:
    for subject, predicate, object_ in rdf_triples:
        # Define Cypher query to add RDF triple
        predicate = predicate.replace(" ", "_")
        query = (
            f"MERGE (s:Resource {{name: '{subject}'}})"
            f"MERGE (o:Resource {{name: '{object_}'}})"
            f"MERGE (s)-[:`{predicate}`]->(o)"
            )
        # Execute the query
        # results = session.execute_read(
        #     lambda tx: tx.run(query).data())
        session.execute_write(execute_query, query)

driver.close()


In [ ]:
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.docstore.document import Document
# from langchain.chains.summarize import load_summarize_chain
# from langchain import PromptTemplate
# from langchain.llms import CTransformers
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler



# # this function is responsible for splitting the data into smaller chunks and convert the data in document format
# def chunks_and_document(txt):

#     text_splitter = CharacterTextSplitter() # text splitter method by default it has chunk_size = 200 and chunk_overlap = 200
#     texts = text_splitter.split_text(txt) # split the text into smaller chunks
#     docs = [Document(page_content=t) for t in texts] # convert the splitted chunks into document format

#     return docs

# # Loading the Llama 2's LLM
# def load_llm():
#     # We instantiate the callback with a streaming stdout handler
#     callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

#     # loading the LLM model
#     # This open source model can be downloaded from here
#     # Their are multiple models available just replace it in place of model and try it.
#     llm = CTransformers(
#         model=r"C:\Users\venky\OneDrive\Desktop\Learnings\Projects\Llama-2-Streamlit-Chatbot\llama-2-7b-chat.ggmlv3.q2_K.bin",
#         model_type="llama",
#          max_new_tokens = 512,
#         temperature = 0.5   )

#     return llm

# # this functions is used for applying the llm model with our document
# def chains_and_response(docs):

#     llm = load_llm()
#     chain = load_summarize_chain(llm,chain_type='map_reduce')

#     return chain.run(docs)

# # Form to accept user's text input for summarization
# result = []
# with st.form('summarize_form', clear_on_submit=True):
#     submitted = st.form_submit_button('Submit')
#     #if submitted and openai_api_key.startswith('sk-'):
#     if submitted:
#         with st.spinner('Calculating...'):
#             docs = chunks_and_document(txt_input)
#             response = chains_and_response(docs)
#             result.append(response)

In [ ]:
# !pip install pytextrank
# !python3 -m spacy download en_core_web_lg

In [ ]:
# import spacy
# import pytextrank

# nlp = spacy.load("en_core_web_lg")
# nlp.add_pipe("textrank")

# print('Original Document Size:',len(processed_text))
# doc = nlp(processed_text)

# for sent in doc._.textrank.summary(limit_phrases=2, limit_sentences=2):
#     print(sent)
#     print('Summary Length:',len(sent))

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Sample user question
user_question = "Natural Language processing is studied by whom?"

# Tokenize the user question
tokens = word_tokenize(user_question)

# Part-of-speech (POS) tagging
tagged_tokens = nltk.pos_tag(tokens)

# Named entity recognition (NER)
named_entities = nltk.ne_chunk(tagged_tokens)

# Identify entities and relationships
entities = []
relationships = []

for entity in named_entities:
    if isinstance(entity, nltk.Tree):
        entities.append(" ".join([word for word, tag in entity.leaves()]))
    else:
        word, tag = entity
        if tag.startswith('NN'):  # Consider only nouns
            entities.append(word)

# Extract constraints
constraints = []
for i, (word, tag) in enumerate(tagged_tokens):
    if word.lower() == 'after' and i < len(tagged_tokens) - 1:
        next_word, next_tag = tagged_tokens[i + 1]
        if next_tag == 'CD':  # If the next word is a cardinal number (e.g., '2019')
            constraints.append(next_word)

print("Entities:", entities)
print("Constraints:", constraints)


Entities: ['Language', 'processing']
Constraints: []
